# Healthcare Analytics: Delta Liquid Clustering Demo


## Overview


This notebook demonstrates the power of **Delta Liquid Clustering** in Oracle AI Data Platform (AIDP) Workbench using a healthcare analytics use case. Liquid clustering automatically optimizes data layout for query performance without requiring manual partitioning or Z-Ordering.

### What is Liquid Clustering?

Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:

- **Automatic optimization**: No manual tuning required
- **Improved query performance**: Faster queries on clustered columns
- **Reduced maintenance**: No need for manual repartitioning
- **Adaptive clustering**: Adjusts as data patterns change

### Use Case: Patient Diagnosis Analytics

We'll analyze patient diagnosis records from a healthcare system. Our clustering strategy will optimize for:

- **Patient-specific queries**: Fast lookups by patient ID
- **Time-based analysis**: Efficient filtering by diagnosis date
- **Diagnosis patterns**: Quick aggregation by diagnosis type

### AIDP Environment Setup

This notebook leverages the existing Spark session in your AIDP environment.

In [ ]:
# Create healthcare catalog and analytics schema

# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS healthcare")

spark.sql("CREATE SCHEMA IF NOT EXISTS healthcare.analytics")

print("Healthcare catalog and analytics schema created successfully!")

Healthcare catalog and analytics schema created successfully!


## Step 2: Create Delta Table with Liquid Clustering

### Table Design

Our `patient_diagnoses` table will store:

- **patient_id**: Unique patient identifier
- **diagnosis_date**: Date of diagnosis
- **diagnosis_code**: ICD-10 diagnosis code
- **diagnosis_description**: Human-readable diagnosis
- **severity_level**: Critical, High, Medium, Low
- **treating_physician**: Physician ID
- **facility_id**: Healthcare facility

### Clustering Strategy

We'll cluster by `patient_id` and `diagnosis_date` because:

- **patient_id**: Patients often have multiple visits, grouping their records together
- **diagnosis_date**: Time-based queries are critical for tracking patient health over time
- This combination optimizes for both individual patient monitoring and temporal health analysis

In [ ]:
# Create Delta table with liquid clustering

# CLUSTER BY defines the columns for automatic optimization

spark.sql("""

CREATE TABLE IF NOT EXISTS healthcare.analytics.patient_diagnoses (

    patient_id STRING,

    diagnosis_date DATE,

    diagnosis_code STRING,

    diagnosis_description STRING,

    severity_level STRING,

    treating_physician STRING,

    facility_id STRING

)

USING DELTA

CLUSTER BY (patient_id, diagnosis_date)

""")

print("Delta table with liquid clustering created successfully!")

print("Clustering will automatically optimize data layout for queries on patient_id and diagnosis_date.")

Delta table with liquid clustering created successfully!
Clustering will automatically optimize data layout for queries on patient_id and diagnosis_date.


## Step 3: Generate Healthcare Sample Data

### Data Generation Strategy

We'll create realistic healthcare diagnosis data including:

- **1,000 patients** with multiple diagnoses over time
- **Common diagnoses**: Diabetes, Hypertension, Asthma, Cancer treatments, etc.
- **Realistic temporal patterns**: Chronic condition management, follow-up visits
- **Multiple facilities**: Hospitals, clinics, urgent care centers

### Why This Data Pattern?

This data simulates real healthcare scenarios where:

- Patients have multiple encounters over time
- Chronic conditions require ongoing monitoring
- Severity levels affect resource allocation
- Facility-level analysis supports operational decisions

In [ ]:
# Generate sample healthcare diagnosis data

# Using fully qualified imports to avoid conflicts

import random

from datetime import datetime, timedelta


# Define healthcare data constants

DIAGNOSES = [

    ("E11.9", "Type 2 diabetes mellitus without complications", "Medium"),

    ("I10", "Essential hypertension", "High"),

    ("J45.909", "Unspecified asthma, uncomplicated", "Medium"),

    ("M54.5", "Low back pain", "Low"),

    ("N39.0", "Urinary tract infection, site not specified", "Medium"),

    ("Z51.11", "Encounter for antineoplastic chemotherapy", "Critical"),

    ("I25.10", "Atherosclerotic heart disease of native coronary artery without angina pectoris", "High"),

    ("F41.9", "Anxiety disorder, unspecified", "Medium"),

    ("M79.3", "Panniculitis, unspecified", "Low"),

    ("Z00.00", "Encounter for general adult medical examination without abnormal findings", "Low")

]



FACILITIES = ["HOSP001", "HOSP002", "CLINIC001", "CLINIC002", "URGENT001"]

PHYSICIANS = ["DR_SMITH", "DR_JOHNSON", "DR_WILLIAMS", "DR_BROWN", "DR_JONES", "DR_GARCIA", "DR_MILLER", "DR_DAVIS"]


# Generate patient diagnosis records

patient_data = []

base_date = datetime(2024, 1, 1)


# Create 1,000 patients with 2-8 diagnoses each

for patient_num in range(1, 1001):

    patient_id = f"PAT{patient_num:04d}"
    
    # Each patient gets 2-8 diagnoses over 12 months

    num_diagnoses = random.randint(2, 8)
    
    for i in range(num_diagnoses):

        # Spread diagnoses over 12 months

        days_offset = random.randint(0, 365)

        diagnosis_date = base_date + timedelta(days=days_offset)
        
        # Select random diagnosis

        diagnosis_code, description, severity = random.choice(DIAGNOSES)
        
        # Select random facility and physician

        facility = random.choice(FACILITIES)

        physician = random.choice(PHYSICIANS)
        
        patient_data.append({

            "patient_id": patient_id,

            "diagnosis_date": diagnosis_date.date(),

            "diagnosis_code": diagnosis_code,

            "diagnosis_description": description,

            "severity_level": severity,

            "treating_physician": physician,

            "facility_id": facility

        })



print(f"Generated {len(patient_data)} patient diagnosis records")

print("Sample record:", patient_data[0])

Generated 4944 patient diagnosis records
Sample record: {'patient_id': 'PAT0001', 'diagnosis_date': datetime.date(2024, 1, 21), 'diagnosis_code': 'M54.5', 'diagnosis_description': 'Low back pain', 'severity_level': 'Low', 'treating_physician': 'DR_JOHNSON', 'facility_id': 'HOSP002'}


## Step 4: Insert Data Using PySpark

### Data Insertion Strategy

We'll use PySpark to:

1. **Create DataFrame** from our generated data
2. **Insert into Delta table** with liquid clustering
3. **Verify the insertion** with a sample query

### Why PySpark for Insertion?

- **Distributed processing**: Handles large datasets efficiently
- **Type safety**: Ensures data integrity
- **Optimization**: Leverages Spark's query optimization
- **Liquid clustering**: Automatically applies clustering during insertion

In [ ]:
# Insert data using PySpark DataFrame operations

# Using fully qualified function references to avoid conflicts


# Create DataFrame from generated data

df_diagnoses = spark.createDataFrame(patient_data)


# Display schema and sample data

print("DataFrame Schema:")

df_diagnoses.printSchema()



print("\nSample Data:")

df_diagnoses.show(5)


# Insert data into Delta table with liquid clustering

# The CLUSTER BY (patient_id, diagnosis_date) will automatically optimize the data layout

df_diagnoses.write.mode("overwrite").saveAsTable("healthcare.analytics.patient_diagnoses")


print(f"\nSuccessfully inserted {df_diagnoses.count()} records into healthcare.analytics.patient_diagnoses")

print("Liquid clustering automatically optimized the data layout during insertion!")

DataFrame Schema:
root
 |-- diagnosis_code: string (nullable = true)
 |-- diagnosis_date: date (nullable = true)
 |-- diagnosis_description: string (nullable = true)
 |-- facility_id: string (nullable = true)
 |-- patient_id: string (nullable = true)
 |-- severity_level: string (nullable = true)
 |-- treating_physician: string (nullable = true)


Sample Data:


+--------------+--------------+---------------------+-----------+----------+--------------+------------------+
|diagnosis_code|diagnosis_date|diagnosis_description|facility_id|patient_id|severity_level|treating_physician|
+--------------+--------------+---------------------+-----------+----------+--------------+------------------+
|         M54.5|    2024-01-21|        Low back pain|    HOSP002|   PAT0001|           Low|        DR_JOHNSON|
|         N39.0|    2024-07-01| Urinary tract inf...|  CLINIC001|   PAT0001|        Medium|          DR_DAVIS|
|        I25.10|    2024-07-02| Atherosclerotic h...|  CLINIC001|   PAT0002|          High|          DR_JONES|
|         F41.9|    2024-12-17| Anxiety disorder,...|    HOSP001|   PAT0002|        Medium|          DR_JONES|
|         M54.5|    2024-05-22|        Low back pain|    HOSP001|   PAT0002|           Low|          DR_DAVIS|
+--------------+--------------+---------------------+-----------+----------+--------------+------------------+
o


Successfully inserted 4944 records into healthcare.analytics.patient_diagnoses
Liquid clustering automatically optimized the data layout during insertion!


## Step 5: Demonstrate Liquid Clustering Benefits

### Query Performance Analysis

Now let's see how liquid clustering improves query performance. We'll run queries that benefit from our clustering strategy:

1. **Patient diagnosis history** (clustered by patient_id)
2. **Time-based analysis** (clustered by diagnosis_date)
3. **Combined patient + time queries** (optimal for our clustering)

### Expected Performance Benefits

With liquid clustering, these queries should be significantly faster because:

- **Data locality**: Related records are physically grouped together
- **Reduced I/O**: Less data needs to be read from disk
- **Automatic optimization**: No manual tuning required

In [ ]:
# Demonstrate liquid clustering benefits with optimized queries


# Query 1: Patient diagnosis history - benefits from patient_id clustering

print("=== Query 1: Patient Diagnosis History ===")

patient_history = spark.sql("""

SELECT patient_id, diagnosis_date, diagnosis_code, diagnosis_description, severity_level

FROM healthcare.analytics.patient_diagnoses

WHERE patient_id = 'PAT0001'

ORDER BY diagnosis_date DESC

""")



patient_history.show()

print(f"Records found: {patient_history.count()}")



# Query 2: Time-based critical diagnoses - benefits from diagnosis_date clustering

print("\n=== Query 2: Recent Critical Diagnoses ===")

recent_critical = spark.sql("""

SELECT diagnosis_date, patient_id, diagnosis_code, diagnosis_description, treating_physician

FROM healthcare.analytics.patient_diagnoses

WHERE diagnosis_date >= '2024-04-01' AND severity_level = 'Critical'

ORDER BY diagnosis_date DESC

""")



recent_critical.show()

print(f"Critical diagnoses found: {recent_critical.count()}")



# Query 3: Combined patient + time query - optimal for our clustering strategy

print("\n=== Query 3: Patient Health Timeline ===")

patient_timeline = spark.sql("""

SELECT patient_id, diagnosis_date, diagnosis_code, severity_level, facility_id

FROM healthcare.analytics.patient_diagnoses

WHERE patient_id LIKE 'PAT001%' AND diagnosis_date >= '2024-03-01'

ORDER BY patient_id, diagnosis_date

""")



patient_timeline.show()

print(f"Timeline records found: {patient_timeline.count()}")

=== Query 1: Patient Diagnosis History ===


+----------+--------------+--------------+---------------------+--------------+
|patient_id|diagnosis_date|diagnosis_code|diagnosis_description|severity_level|
+----------+--------------+--------------+---------------------+--------------+
|   PAT0001|    2024-07-01|         N39.0| Urinary tract inf...|        Medium|
|   PAT0001|    2024-01-21|         M54.5|        Low back pain|           Low|
+----------+--------------+--------------+---------------------+--------------+



Records found: 2

=== Query 2: Recent Critical Diagnoses ===


+--------------+----------+--------------+---------------------+------------------+
|diagnosis_date|patient_id|diagnosis_code|diagnosis_description|treating_physician|
+--------------+----------+--------------+---------------------+------------------+
|    2024-12-31|   PAT0645|        Z51.11| Encounter for ant...|          DR_BROWN|
|    2024-12-30|   PAT0648|        Z51.11| Encounter for ant...|          DR_BROWN|
|    2024-12-30|   PAT0808|        Z51.11| Encounter for ant...|          DR_SMITH|
|    2024-12-29|   PAT0155|        Z51.11| Encounter for ant...|         DR_GARCIA|
|    2024-12-29|   PAT0878|        Z51.11| Encounter for ant...|          DR_JONES|
|    2024-12-27|   PAT0073|        Z51.11| Encounter for ant...|       DR_WILLIAMS|
|    2024-12-26|   PAT0392|        Z51.11| Encounter for ant...|          DR_DAVIS|
|    2024-12-26|   PAT0684|        Z51.11| Encounter for ant...|          DR_BROWN|
|    2024-12-25|   PAT0203|        Z51.11| Encounter for ant...|          DR

Critical diagnoses found: 349

=== Query 3: Patient Health Timeline ===


+----------+--------------+--------------+--------------+-----------+
|patient_id|diagnosis_date|diagnosis_code|severity_level|facility_id|
+----------+--------------+--------------+--------------+-----------+
|   PAT0010|    2024-04-25|        Z51.11|      Critical|    HOSP002|
|   PAT0010|    2024-05-22|         M54.5|           Low|  URGENT001|
|   PAT0010|    2024-07-13|         M79.3|           Low|    HOSP001|
|   PAT0010|    2024-07-16|        Z51.11|      Critical|    HOSP001|
|   PAT0010|    2024-09-05|        Z00.00|           Low|    HOSP001|
|   PAT0010|    2024-11-26|        Z00.00|           Low|    HOSP002|
|   PAT0011|    2024-03-30|        I25.10|          High|  CLINIC001|
|   PAT0011|    2024-09-23|        I25.10|          High|    HOSP002|
|   PAT0012|    2024-05-28|         F41.9|        Medium|    HOSP001|
|   PAT0012|    2024-08-01|        I25.10|          High|    HOSP002|
|   PAT0012|    2024-09-14|        Z51.11|      Critical|  URGENT001|
|   PAT0012|    2024

Timeline records found: 42


## Step 6: Analyze Clustering Effectiveness

### Understanding the Impact

Let's examine how liquid clustering has organized our data and analyze some aggregate statistics to demonstrate the healthcare insights possible with this optimized structure.

### Key Analytics

- **Diagnosis frequency** and prevalence analysis
- **Severity distribution** across facilities and physicians
- **Physician workload** and patient load analysis
- **Temporal patterns** in healthcare utilization

In [ ]:
# Analyze clustering effectiveness and healthcare insights


# Diagnosis frequency analysis

print("=== Diagnosis Frequency Analysis ===")

diagnosis_freq = spark.sql("""

SELECT diagnosis_code, diagnosis_description, COUNT(*) as frequency,

       ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage

FROM healthcare.analytics.patient_diagnoses

GROUP BY diagnosis_code, diagnosis_description

ORDER BY frequency DESC

""")



diagnosis_freq.show(truncate=False)


# Severity distribution by facility

print("\n=== Severity Distribution by Facility ===")

severity_by_facility = spark.sql("""

SELECT facility_id, severity_level, COUNT(*) as count

FROM healthcare.analytics.patient_diagnoses

GROUP BY facility_id, severity_level

ORDER BY facility_id, severity_level

""")



severity_by_facility.show()


# Physician workload analysis

print("\n=== Physician Workload Analysis ===")

physician_workload = spark.sql("""

SELECT treating_physician, COUNT(*) as total_diagnoses,

       COUNT(DISTINCT patient_id) as unique_patients,

       ROUND(AVG(CASE WHEN severity_level = 'Critical' THEN 1 ELSE 0 END), 3) as critical_case_ratio

FROM healthcare.analytics.patient_diagnoses

GROUP BY treating_physician

ORDER BY total_diagnoses DESC

""")



physician_workload.show()


# Monthly diagnosis trends

print("\n=== Monthly Diagnosis Trends ===")

monthly_trends = spark.sql("""

SELECT DATE_FORMAT(diagnosis_date, 'yyyy-MM') as month,

       COUNT(*) as total_diagnoses,

       COUNT(DISTINCT patient_id) as unique_patients,

       ROUND(AVG(CASE WHEN severity_level = 'Critical' THEN 1 ELSE 0 END), 3) as critical_rate

FROM healthcare.analytics.patient_diagnoses

GROUP BY DATE_FORMAT(diagnosis_date, 'yyyy-MM')

ORDER BY month

""")



monthly_trends.show()

=== Diagnosis Frequency Analysis ===


+--------------+-------------------------------------------------------------------------------+---------+----------+
|diagnosis_code|diagnosis_description                                                          |frequency|percentage|
+--------------+-------------------------------------------------------------------------------+---------+----------+
|I10           |Essential hypertension                                                         |511      |10.34     |
|E11.9         |Type 2 diabetes mellitus without complications                                 |503      |10.17     |
|Z00.00        |Encounter for general adult medical examination without abnormal findings      |503      |10.17     |
|I25.10        |Atherosclerotic heart disease of native coronary artery without angina pectoris|501      |10.13     |
|N39.0         |Urinary tract infection, site not specified                                    |499      |10.09     |
|M79.3         |Panniculitis, unspecified               

+-----------+--------------+-----+
|facility_id|severity_level|count|
+-----------+--------------+-----+
|  CLINIC001|      Critical|   91|
|  CLINIC001|          High|  191|
|  CLINIC001|           Low|  300|
|  CLINIC001|        Medium|  409|
|  CLINIC002|      Critical|   94|
|  CLINIC002|          High|  204|
|  CLINIC002|           Low|  304|
|  CLINIC002|        Medium|  385|
|    HOSP001|      Critical|   90|
|    HOSP001|          High|  194|
|    HOSP001|           Low|  319|
|    HOSP001|        Medium|  411|
|    HOSP002|      Critical|  102|
|    HOSP002|          High|  211|
|    HOSP002|           Low|  268|
|    HOSP002|        Medium|  387|
|  URGENT001|      Critical|   96|
|  URGENT001|          High|  212|
|  URGENT001|           Low|  295|
|  URGENT001|        Medium|  381|
+-----------+--------------+-----+


=== Physician Workload Analysis ===


+------------------+---------------+---------------+-------------------+
|treating_physician|total_diagnoses|unique_patients|critical_case_ratio|
+------------------+---------------+---------------+-------------------+
|          DR_SMITH|            656|            473|              0.101|
|          DR_JONES|            650|            479|              0.098|
|          DR_DAVIS|            645|            470|              0.076|
|          DR_BROWN|            634|            473|               0.09|
|       DR_WILLIAMS|            628|            468|                0.1|
|         DR_MILLER|            606|            449|              0.102|
|         DR_GARCIA|            572|            451|              0.098|
|        DR_JOHNSON|            553|            422|              0.101|
+------------------+---------------+---------------+-------------------+


=== Monthly Diagnosis Trends ===


+-------+---------------+---------------+-------------+
|  month|total_diagnoses|unique_patients|critical_rate|
+-------+---------------+---------------+-------------+
|2024-01|            419|            350|          0.1|
|2024-02|            392|            329|        0.092|
|2024-03|            419|            352|         0.11|
|2024-04|            404|            331|        0.079|
|2024-05|            420|            343|        0.112|
|2024-06|            431|            352|        0.095|
|2024-07|            399|            333|        0.098|
|2024-08|            377|            306|        0.106|
|2024-09|            421|            344|        0.093|
|2024-10|            441|            361|        0.084|
|2024-11|            414|            349|        0.106|
|2024-12|            407|            345|        0.074|
+-------+---------------+---------------+-------------+



## Step 7: Train Healthcare Patient Readmission Prediction Model

### Machine Learning for Healthcare Business Improvement

Now we'll train a machine learning model to predict patient readmission risk. This model can help healthcare organizations:

- **Identify high-risk patients** for readmission prevention
- **Optimize resource allocation** for patient care management
- **Improve care coordination** and intervention strategies
- **Reduce healthcare costs** associated with preventable readmissions

### Model Approach

We'll use a **Random Forest Classifier** to predict 30-day readmission risk based on:

- Patient diagnosis history and severity patterns
- Facility and physician utilization patterns
- Temporal patterns in healthcare encounters
- Diagnosis frequency and complexity indicators

### Business Impact

- **Cost Reduction**: Prevent expensive readmission episodes
- **Quality Improvement**: Better patient outcomes and satisfaction
- **Operational Efficiency**: Targeted care management programs
- **Regulatory Compliance**: Improved quality metrics and reporting

In [ ]:
# Prepare data for machine learning - create patient-level readmission features

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
import pyspark.sql.functions as F

# Create patient-level features for readmission prediction
patient_features = spark.sql("""
SELECT 
    patient_id,
    COUNT(*) as total_diagnoses,
    COUNT(DISTINCT diagnosis_code) as unique_diagnoses,
    ROUND(AVG(CASE WHEN severity_level = 'Critical' THEN 1 
                    WHEN severity_level = 'High' THEN 0.75 
                    WHEN severity_level = 'Medium' THEN 0.5 
                    ELSE 0.25 END), 3) as avg_severity_score,
    COUNT(DISTINCT facility_id) as facilities_used,
    COUNT(DISTINCT treating_physician) as physicians_seen,
    COUNT(DISTINCT DATE_FORMAT(diagnosis_date, 'yyyy-MM')) as active_months,
    DATEDIFF(CURRENT_DATE(), MAX(diagnosis_date)) as days_since_last_visit,
    DATEDIFF(CURRENT_DATE(), MIN(diagnosis_date)) as patient_tenure_days,
    ROUND(AVG(DATEDIFF(diagnosis_date, lag_date)), 2) as avg_days_between_visits,
    -- Readmission risk factors
    CASE WHEN COUNT(*) > 6 THEN 1 ELSE 0 END as high_visit_frequency,
    CASE WHEN COUNT(DISTINCT diagnosis_code) > 4 THEN 1 ELSE 0 END as complex_case,
    CASE WHEN AVG(CASE WHEN severity_level = 'Critical' THEN 1 
                       WHEN severity_level = 'High' THEN 0.75 
                       WHEN severity_level = 'Medium' THEN 0.5 
                       ELSE 0.25 END) > 0.6 THEN 1 ELSE 0 END as high_severity_patient,
    -- Target: Readmission risk (simulated based on risk factors)
    CASE WHEN 
        COUNT(*) > 6 OR 
        COUNT(DISTINCT diagnosis_code) > 4 OR 
        AVG(CASE WHEN severity_level = 'Critical' THEN 1 
                 WHEN severity_level = 'High' THEN 0.75 
                 WHEN severity_level = 'Medium' THEN 0.5 
                 ELSE 0.25 END) > 0.6 OR
        COUNT(DISTINCT facility_id) > 2
    THEN 1 ELSE 0 END as readmission_risk
FROM (select *, LAG(diagnosis_date) OVER (PARTITION BY patient_id ORDER BY diagnosis_date) lag_date from healthcare.analytics.patient_diagnoses)
GROUP BY patient_id
""")

# Fill null values from window functions
patient_features = patient_features.fillna(30, subset=['avg_days_between_visits'])

print(f"Created patient readmission features for {patient_features.count()} patients")
patient_features.groupBy("readmission_risk").count().show()

Created patient readmission features for 1000 patients


+----------------+-----+
|readmission_risk|count|
+----------------+-----+
|               1|  804|
|               0|  196|
+----------------+-----+



In [ ]:
# Feature engineering for readmission prediction

# Assemble features for the model
feature_cols = ["total_diagnoses", "unique_diagnoses", "avg_severity_score", "facilities_used", 
                "physicians_seen", "active_months", "days_since_last_visit", "patient_tenure_days", 
                "avg_days_between_visits", "high_visit_frequency", "complex_case", "high_severity_patient"]

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

# Scale features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

# Create and train the model
rf = RandomForestClassifier(
    labelCol="readmission_risk", 
    featuresCol="scaled_features",
    numTrees=100,
    maxDepth=10
)

# Create pipeline
pipeline = Pipeline(stages=[assembler, scaler, rf])

# Split data
train_data, test_data = patient_features.randomSplit([0.8, 0.2], seed=42)

print(f"Training set: {train_data.count()} patients")
print(f"Test set: {test_data.count()} patients")

Training set: 838 patients


Test set: 162 patients


In [ ]:
# Train the patient readmission prediction model

print("Training patient readmission prediction model...")
model = pipeline.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol="readmission_risk", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)

print(f"Model AUC: {auc:.4f}")

# Show prediction results
predictions.select("patient_id", "total_diagnoses", "avg_severity_score", "readmission_risk", "prediction", "probability").show(10)

# Calculate confusion matrix
confusion_matrix = predictions.groupBy("readmission_risk", "prediction").count()
confusion_matrix.show()

Training patient readmission prediction model...


Model AUC: 1.0000


+----------+---------------+------------------+----------------+----------+--------------------+
|patient_id|total_diagnoses|avg_severity_score|readmission_risk|prediction|         probability|
+----------+---------------+------------------+----------------+----------+--------------------+
|   PAT0003|              2|             0.750|               1|       1.0|[0.03047619047619...|
|   PAT0007|              3|             0.667|               1|       1.0|         [0.01,0.99]|
|   PAT0009|              2|             0.625|               1|       1.0|           [0.0,1.0]|
|   PAT0014|              3|             0.583|               0|       0.0|[0.97104166666666...|
|   PAT0020|              7|             0.536|               1|       1.0|           [0.0,1.0]|
|   PAT0024|              5|             0.600|               1|       1.0|           [0.0,1.0]|
|   PAT0030|              4|             0.750|               1|       1.0|         [0.01,0.99]|
|   PAT0036|              2|  

+----------------+----------+-----+
|readmission_risk|prediction|count|
+----------------+----------+-----+
|               0|       0.0|   31|
|               1|       1.0|  131|
+----------------+----------+-----+



In [ ]:
# Model interpretation and business insights

# Feature importance (approximate)
rf_model = model.stages[-1]
feature_importance = rf_model.featureImportances
feature_names = feature_cols

print("=== Feature Importance for Readmission Prediction ===")
for name, importance in zip(feature_names, feature_importance):
    print(f"{name}: {importance:.4f}")

# Business impact analysis
print("\n=== Business Impact Analysis ===")

# Calculate potential impact of readmission prediction
high_risk_predictions = predictions.filter("prediction = 1")
patients_at_risk = high_risk_predictions.count()
total_test_patients = test_data.count()

print(f"Total test patients: {total_test_patients}")
print(f"Patients predicted as high readmission risk: {patients_at_risk}")
print(f"Percentage flagged for intervention: {(patients_at_risk/total_test_patients)*100:.1f}%")

# Calculate cost savings potential
avg_readmission_cost = 15000  # Estimated cost per readmission episode
intervention_success_rate = 0.3  # 30% reduction in readmissions with interventions
avg_intervention_cost = 2000  # Cost per intervention program

prevented_readmissions = patients_at_risk * intervention_success_rate
cost_savings = prevented_readmissions * avg_readmission_cost
total_intervention_cost = patients_at_risk * avg_intervention_cost
net_savings = cost_savings - total_intervention_cost

print(f"\nEstimated cost per readmission: ${avg_readmission_cost:,}")
print(f"Estimated intervention success rate: {intervention_success_rate*100:.0f}%")
print(f"Potential readmissions prevented: {prevented_readmissions:.0f}")
print(f"Potential cost savings: ${cost_savings:,.0f}")
print(f"Total intervention cost: ${total_intervention_cost:,.0f}")
print(f"Net savings: ${net_savings:,.0f}")

# Quality improvement metrics
print("\n=== Quality Improvement Impact ===")
print(f"Reduction in hospital readmission rate: {(prevented_readmissions/total_test_patients)*100:.1f}%")
print(f"Improvement in patient outcomes for {patients_at_risk} high-risk patients")

# Accuracy metrics
accuracy = predictions.filter("readmission_risk = prediction").count() / predictions.count()
precision = predictions.filter("prediction = 1 AND readmission_risk = 1").count() / predictions.filter("prediction = 1").count() if predictions.filter("prediction = 1").count() > 0 else 0
recall = predictions.filter("prediction = 1 AND readmission_risk = 1").count() / predictions.filter("readmission_risk = 1").count() if predictions.filter("readmission_risk = 1").count() > 0 else 0

print(f"\nModel Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"AUC: {auc:.4f}")

=== Feature Importance for Readmission Prediction ===
total_diagnoses: 0.1135
unique_diagnoses: 0.0464
avg_severity_score: 0.1588
facilities_used: 0.3835
physicians_seen: 0.0298
active_months: 0.0505
days_since_last_visit: 0.0124
patient_tenure_days: 0.0155
avg_days_between_visits: 0.0336
high_visit_frequency: 0.0027
complex_case: 0.0174
high_severity_patient: 0.1360

=== Business Impact Analysis ===


Total test patients: 162
Patients predicted as high readmission risk: 131
Percentage flagged for intervention: 80.9%

Estimated cost per readmission: $15,000
Estimated intervention success rate: 30%
Potential readmissions prevented: 39
Potential cost savings: $589,500
Total intervention cost: $262,000
Net savings: $327,500

=== Quality Improvement Impact ===
Reduction in hospital readmission rate: 24.3%
Improvement in patient outcomes for 131 high-risk patients



Model Performance:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
AUC: 1.0000


## Key Takeaways: Delta Liquid Clustering + ML in AIDP

### What We Demonstrated

1. **Automatic Optimization**: Created a table with `CLUSTER BY (patient_id, diagnosis_date)` and let Delta automatically optimize data layout

2. **Performance Benefits**: Queries on clustered columns (patient_id, diagnosis_date) are significantly faster due to data locality

3. **Zero Maintenance**: No manual partitioning, bucketing, or Z-Ordering required - Delta handles it automatically

4. **Machine Learning Integration**: Trained a patient readmission prediction model using the optimized data

5. **Real-World Use Case**: Healthcare analytics where patient risk prediction and care management are critical

### AIDP Advantages

- **Unified Analytics**: Seamlessly integrates data optimization with ML
- **Governance**: Catalog and schema isolation for healthcare data
- **Performance**: Optimized for both analytical queries and ML training
- **Scalability**: Handles healthcare-scale data volumes effortlessly

### Business Benefits for Healthcare

1. **Cost Reduction**: Prevent expensive readmission episodes through early intervention
2. **Quality Improvement**: Better patient outcomes and care coordination
3. **Operational Efficiency**: Targeted care management for high-risk patients
4. **Regulatory Compliance**: Improved quality metrics and reporting
5. **Patient Satisfaction**: Proactive care reduces negative experiences

### Best Practices for Healthcare Analytics

1. **Choose clustering columns** based on your most common query patterns
2. **Start with 1-4 columns** - too many can reduce effectiveness
3. **Consider cardinality** - high-cardinality columns work best
4. **Monitor and adjust** as query patterns evolve
5. **Combine with ML** for predictive analytics and automation

### Next Steps

- Explore other AIDP ML features like AutoML
- Try liquid clustering with different column combinations
- Scale up to larger healthcare datasets
- Integrate with real EHR systems and patient monitoring
- Deploy models for real-time readmission risk monitoring

This notebook demonstrates how Oracle AI Data Platform makes advanced healthcare analytics accessible while maintaining enterprise-grade performance and governance.